## Import Dependencies and Setup Enviroment

In [30]:
import os
import openai
import json
import time
from dotenv import load_dotenv
from langchain.llms import OpenAI

In [31]:
_ = load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

## Generate Seed Data via LLM

In [32]:
def generate_home_listings(prompt):
    system_prompt = """
    You are a real estate listing assistant specializing in crafting compelling and informative home descriptions. Your task is to generate real estate listings in a structured format based on user-provided details.
    
    Each listing should follow this format:
    
    Neighborhood: [Neighborhood Name]
    Price: [$Price]
    Bedrooms: [Number of Bedrooms]
    Bathrooms: [Number of Bathrooms]
    House Size: [Size in sqft]
    
    Description:
    Provide a captivating yet concise property description, emphasizing key selling points such as architectural style, unique features, energy efficiency, or location benefits. Use persuasive language to make the home appealing to potential buyers. Highlight amenities, natural lighting, interior design elements, and lifestyle advantages.
    
    Neighborhood Description:
    Offer a brief, engaging description of the neighborhood. Highlight aspects such as community atmosphere, parks, restaurants, transportation, and local attractions. Focus on features that would be appealing to potential buyers considering the area.
    
    Maintain a professional and inviting tone, ensuring the listing feels aspirational yet grounded in reality. Avoid exaggeration while emphasizing the home’s best features.
    """
    
    try:
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": prompt
          }
          ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"An error occurred: {e}" 

def generate_multiple_listings(prompts):
    listings = []

    for i, prompt in enumerate(prompts):
        print(f"Generating listing {i+1}/{len(prompts)}...")
        listing = generate_home_listings(prompt)
        
        if "An error occurred" not in listing:
            listings.append(listing)
        else:
            print(f"Error in listing {i+1}: {listing}")

        time.sleep(1)  

    with open("home_listings.json", "w") as f:
        json.dump(listings, f, indent=4)

    print(f"Saved {len(listings)} listings to home_listings.json")

    return listings

In [34]:
prompts = [
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 4,000 sqft in Baltimore, priced at $400,000. The home has a parking pad and is near the park.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,200 sqft in Washington DC, priced at $600,000. The home has a back patio and has street parking.",
    "Generate a listing for a 6-bedroom, 3 bathroom house, with 8,000 sqft in Dallas, priced at $500,000. The home has a large backyard and has three garages.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 400 sqft in New York, priced at $800,000. The apartment has bathroom kitchen and features a window.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,100 sqft in Baltimore, priced at $205,000. The home has a view of the harbor and is walking distance to the stadiums.",
    "Generate a listing for a 2-bedroom, 1 bathroom house, with 1,100 sqft in San Francisco, priced at $812,000. The home has is close to public transit and is near a park.",
    "Generate a listing for a 3-bedroom, 1 bathroom apartment, with 1,100 sqft in Chicago, priced at $700,000. The home is an apartment and has a view of the river.",
    "Generate a listing for a 3-bedroom, 2 bathroom house, with 2,100 sqft in New York, priced at $500,000. The home is in upstate New York and is near a wings restaurant.",
    "Generate a listing for a 3-bedroom, 2 bathroom house, with 1,100 sqft in Baltimore, priced at $190,000. The home near Canton square and has many dining options near it.",
    "Generate a listing for a 2-bedroom, 2 bathroom house, with 1,700 sqft in Dallas, priced at $195,000. The home is near the PGA headquarters and offers many opportunities to golf.",
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 1,900 sqft in Eldersburg, priced at $450,000. The home is in a great school district.",
    "Generate a listing for a 4-bedroom, 2 bathroom house, with 1,700 sqft in Phoenix, priced at $400,000. The home is in the desert, has great views and is in a valley.",
    "Generate a listing for a 6-bedroom, 2 bathroom house, with 6,700 sqft in Dallas, priced at $600,000. The home is in the suberbs of Dallas and is only a 25 min drive from the city.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 900 sqft in New York, priced at $800,000. The home is an apartment and is a 15 min walk from the park.",
    "Generate a listing for a 1-bedroom, 1 bathroom apartment, with 1,200 sqft in Miami, priced at $800,000. The home is an apartment and is a 15 min walk from the beach.",
]

listings = generate_multiple_listings(prompts)
print(json.dumps(listings[:2], indent=4))

Generating listing 1/15...
Generating listing 2/15...
Generating listing 3/15...
Generating listing 4/15...
Generating listing 5/15...
Generating listing 6/15...
Generating listing 7/15...
Generating listing 8/15...
Generating listing 9/15...
Generating listing 10/15...
Generating listing 11/15...
Generating listing 12/15...
Generating listing 13/15...
Generating listing 14/15...
Generating listing 15/15...
Saved 15 listings to home_listings.json
[
    "Neighborhood: Baltimore\nPrice: $400,000\nBedrooms: 4\nBathrooms: 2\nHouse Size: 4,000 sqft\n\nDescription:\nWelcome to your spacious and charming 4-bedroom, 2-bathroom home in the heart of Baltimore! This home boasts a generous 4,000 sqft of living space designed for comfortable and stylish living. The property features a convenient parking pad, ensuring hassle-free parking for you and your guests. Enjoy the tranquility and beauty of the nearby park, perfect for leisurely strolls and outdoor activities. \n\nInside, the house exudes a p